<a href="https://colab.research.google.com/github/rhaagsma/Projeto_Final_Marvel/blob/insight-2/InsightsCharacteresComicsEvents.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import csv
import sqlite3
import pandas as pd
from datetime import datetime
import matplotlib.pyplot as plt
import ast
import seaborn as sns

In [ ]:
pd.set_option('display.max_colwidth', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

### Top 10 personagens que estão em mais comics

In [ ]:
con = sqlite3.connect('banco.db')
query = """
      SELECT
            name,
            comics_available
      FROM characters
      ORDER BY comics_available DESC
      LIMIT 10;
"""
df = pd.read_sql_query(query, con)
con.close()

plt.figure(figsize=(12, 6))
plt.bar(df['name'], df['comics_available'])
plt.xlabel("Nome do Personagem")
plt.ylabel("Quantidade de Quadrinhos Disponíveis")
plt.title("Top 10 Personagens com Mais Quadrinhos")
plt.xticks(rotation=45, ha='right')
plt.tight_layout()

for i, v in enumerate(df['comics_available']):
    plt.text(i, v + 25, f"{v}", ha='center')

plt.show()

### Quantidade de participações de personagem em quadrinhos por em evento

In [ ]:
conn = sqlite3.connect('banco.db')

query = """
        SELECT
            e.id AS id,
            e.title AS evento,
            ch.name AS personagem,
            COUNT(*) AS total_participacoes,
            e.amount_comics AS quantidade_comics,
            ROW_NUMBER() OVER (PARTITION BY e.title ORDER BY COUNT(*) DESC) AS posicao
        FROM comics c
        JOIN characters_comics cc ON cc.comic_id = c.id
        JOIN characters ch ON cc.character_id = ch.id
        JOIN comics_events ce ON ce.comic_id = c.id
        JOIN events e ON ce.event_id = e.id
        GROUP BY e.id, e.title, ch.name
        ORDER BY quantidade_comics DESC
"""

df = pd.read_sql_query(query, conn)
conn.close()
df.head()

### Top 10 eventos com mais quadrinhos

In [ ]:
result_df = (df[df['posicao'] == 1]).reset_index(drop=True)
colunas_desejadas = ['evento', 'quantidade_comics']
df_eventos = result_df[colunas_desejadas].head(10)
df_eventos

### Top 10 personagens com mais participações no top 5 de eventos com mais quadrinhos

In [ ]:
result_df = (df[df['posicao'].isin(range(1,11))]).reset_index(drop=True)
colunas_desejadas = ['evento', 'personagem', 'total_participacoes', 'quantidade_comics']
df_comics = result_df[colunas_desejadas].head(50)
df_comics

### Quantidade de quadrinhos de um personagem por décadas


In [ ]:
conn = sqlite3.connect('banco.db')

query = """
    SELECT
        ch.id AS id,
        (c.year / 10) * 10 AS decade,
        ch.name AS personagem,
        COUNT(*) as participacoes,
        ROW_NUMBER() OVER (PARTITION BY (c.year / 10) * 10  ORDER BY COUNT(*) DESC) AS posicao
    FROM comics c
    JOIN characters_comics cc ON cc.comic_id == c.id
    JOIN characters ch ON cc.character_id == ch.id
    WHERE decade != 0
    GROUP BY ch.id, ch.name, decade
    ORDER BY decade, participacoes DESC
"""
df_decade = pd.read_sql_query(query, conn)

conn.close()

df_decade.head()

### Personagens como mais participações em quadrinhos por década

In [ ]:
df_cd = (df_decade[df_decade['posicao'] == 1]).reset_index(drop=True)
colunas_desejadas = ['decade', 'personagem', 'participacoes',]
df_cd[colunas_desejadas]

### Décadas com mais quadrinhos

In [ ]:
conn = sqlite3.connect('banco.db')

query = """
    SELECT
        (c.year / 10) * 10 AS decade,
        COUNT(*) as quantidade_comics
    FROM comics c
    WHERE decade != 0
    GROUP BY decade
    ORDER BY quantidade_comics DESC
"""
decades = pd.read_sql_query(query, conn)

conn.close()
decades